In [1]:
# !pip install torchvision

In [2]:
# !pip install torch

# Mask R-CNN 모델을 이용한 객체 감지

In [3]:
# 필요한 라이브러리 임포트
import torch
from torchvision.models.detection import maskrcnn_resnet50_fpn
from torchvision.transforms import functional as F
from PIL import Image

In [4]:
# 사전 학습 마스크 R-CNN 로딩
model = maskrcnn_resnet50_fpn(pretrained=True)

# 모델 평가
model.eval()

/opt/anaconda3/envs/tf_env/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/envs/tf_env/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MaskRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=MaskRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


MaskRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(in

In [5]:
# 이미지 다운로드
!wget http://images.cocodataset.org/val2017/000000039769.jpg -o coco.jpg
!wget https://ultralytics.com/images/bus.jpg -o bus.jpg

In [6]:
# 이미지 오픈
from PIL import Image
image = Image.open('bus.jpg').convert('RGB')

# 이미지 전처리
image = F.to_tensor(image) # 텐서변환
image = image.unsqueeze(0) # 차원추가

# 예측
with torch.no_grad():
    prediction = model(image)

    boxes = prediction[0]['boxes']

UnidentifiedImageError: cannot identify image file 'bus.jpg'

In [ ]:
from torchvision.utils import draw_bounding_boxes
draw_bounding_boxes(image,boxes,5)

# DETR 모델을 이용한 객체감지

In [ ]:
# !pip install timm -U

In [ ]:
# 라이브러리 임포트
# Load model directly
import torch
from PIL import Image
import requests
from transformers import DetrImageProcessor, DetrForObjectDetection

In [ ]:
# 이미지 오픈
image = Image.open('coco.jpg').convert('RGB')

In [ ]:
# 모델 로드
processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50")

In [ ]:
# 이미지 전처리
inputs = processor(images=image, return_tensors="pt")

# 예측
predictions = model(**inputs)

# 바운딩 박스 좌표
predictions['pred_boxes']

# DETR 파이프라인을 이용한 객체감지

In [ ]:
# 파이프 라인 로딩
from transformers import pipeline

pipe = pipeline("object-detection", model="facebook/detr-resnet-50")

In [ ]:
# 이미지 오픈
image = Image.open('coco.jpg').convert('RGB')

pipe.predict(image)

# 코코 데이터셋 가져오기

In [ ]:
# !pip install fiftyone

In [ ]:
import fiftyone

dataset = fiftyone.zoo.load_zoo_dataset("coco-2017")
dataset

In [ ]:
dataset = fiftyone.zoo.load_zoo_dataset(
    "coco-2017",
    split="validation",
    label_types=["detections", "segmentations"],
    classes=["person", "car"],
    max_samples=50,
)

In [ ]:
# Visualize the dataset in the FiftyOne App
session = fiftyone.launch_app(dataset)

# YOLO를 이용한 객체감지

In [ ]:
# !pip install ultralytics

In [ ]:
# 예측
with torch.no_grad():
    prediction = model(image)

In [ ]:
from ultralytics import YOLO
# Load a COCO-pretrained YOLOv8n model
model= YOLO("yolov8n.pt")
# Display model information (optional)
model.info()
# Train the model on the COCO8 example dataset for 100 epochs
results = model.train(data="coco8.yaml", epochs=100, imgsz=640)
# Run inference with the YOLOv8n model on the '20.jpg' image
results= model("https://ultralytics.com/images/bus.jpg")

# Tiny YOLO를 이용한 객체 탐지

In [ ]:
# Load model directly
from transformers import AutoImageProcessor, AutoModelForObjectDetection

processor = AutoImageProcessor.from_pretrained("hustvl/yolos-tiny")
model = AutoModelForObjectDetection.from_pretrained("hustvl/yolos-tiny")

In [ ]:
from torchvision.transforms import functional as F

# 이미지 가져오기
image = Image.open('coco.jpg').convert('RGB')
# 예측

# 이미지 전처리
image = F.to_tensor(image) # 텐서변환
image = image.unsqueeze(0) # 차원추가

# 예측
with torch.no_grad():
    prediction = model(image)

prediction

In [ ]:
# 이미지 가져오기
image = Image.open('coco.jpg').convert('RGB')

# 전처리기, 객체탐지기 설정
from transformers import AutoImageProcessor, AutoModelForObjectDetection

processor = AutoImageProcessor.from_pretrained("hustvl/yolos-tiny")
model = AutoModelForObjectDetection.from_pretrained("hustvl/yolos-tiny")

# 전처리 및 예측
inputs = processor(images=image, return_tensors="pt") #전처리
outputs = model(**inputs) #예측

# 출력값 포매팅
target_sizes = torch.tensor([image.size[::-1]]) # 높이, 폭인 텐서로 변환
results = processor.post_process_object_detection(outputs, threshold=0.9, target_sizes=target_sizes)[0] # 사이즈 변환

# 객체 감지 결과 반복
for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    box = [round(i, 2) for i in box.tolist()] # 소수 둘째자리 반올림
    print(f"Detected {model.config.id2label[label.item()]} with confidence ")
    print(f"{round(score.item(), 3)} at location {box}")

# roboflow 이용해서 데이터 증강

In [ ]:
# !pip install roboflow

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="5vRdkgspDTVwQk0t4A41")
project = rf.workspace("ssong-w9vsb").project("hard-hat-sample-oocvp")
version = project.version(2)
dataset = version.download("yolov8")